# **Interior Segmentation based on Scenescript**


---



A short notebook to run scenescript by meta and test it with data generated by Trellis image to 3D model.

Run on a A100 on Colab.

https://github.com/facebookresearch/scenescript

https://github.com/microsoft/TRELLIS



---
## Install the dependencies and clone the git


In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:09
🔁 Restarting kernel...


In [ ]:
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [ ]:
!git clone https://github.com/facebookresearch/scenescript.git

Cloning into 'scenescript'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 51 (delta 16), reused 41 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (51/51), 1.33 MiB | 1000.00 KiB/s, done.
Resolving deltas: 100% (16/16), done.


In [ ]:
%cd scenescript

/content/scenescript


In [ ]:
!mamba env update -n base -f environment.yaml

Channels:
 - nvidia/label/cuda-12.1.0
 - pytorch
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: \ | / - \ | done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.2
    latest version: 25.3.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



pytorch-2.4.0        | 1.35 GB   | :   0% 0/1 [00:00<?, ?it/s]
nsight-compute-2023. | 770.2 MB  | :   0% 0/1 [00:00<?, ?it/s]

libcublas-static-12. | 355.1 MB  | :   0% 0/1 [00:00<?, ?it/s]


libcublas-12.1.0.26  | 329.0 MB  | :   0% 0/1 [00:00<?, ?it/s]



torchtriton-3.0.0    | 233.7 MB  | :   0% 0/1 [00:00<?, ?it/s]




libcufft-static-11.0 | 190.0 MB  | :   0% 0/1 [00:00<?, ?it/s]





libcusparse-static-1 | 169.2 MB  | :   0% 0/1 [00:00<?, ?it/s]






libcusparse-12.0.2.5 | 163.0 MB  | :   0% 0/1 [00:00<?, ?it/s]







libcusparse-dev-12.0 | 162.7 MB  | :   0% 0/1 [00:00<?, ?it/s]








libnpp-12.0.2.50     | 139.8 MB  | :   0% 0/1 [00:00<?, ?it/



---

## Import the dependencies and define utility methods

In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation

import plotly.graph_objects as go

from src.data.language_sequence import LanguageSequence
from src.data.point_cloud import PointCloud
from src.networks.scenescript_model import SceneScriptWrapper

In [ ]:
UNIT_CUBE_VERTICES = (
    np.array(
        [
            (1, 1, 1),
            (1, 1, -1),
            (1, -1, 1),
            (1, -1, -1),
            (-1, 1, 1),
            (-1, 1, -1),
            (-1, -1, 1),
            (-1, -1, -1),
        ]
    )
    * 0.5
)


UNIT_CUBE_LINES_IDXS = np.array(
    [
        [0, 1],
        [0, 2],
        [0, 4],
        [1, 3],
        [1, 5],
        [2, 3],
        [2, 6],
        [3, 7],
        [4, 5],
        [4, 6],
        [5, 7],
        [6, 7],
    ]
)


PLOTTING_COLORS = {
    "wall": "#FBFAF5",
    "door": "#F7C59F",
    "window": "#53F4FF",
    "bbox": "#CC3FD1",
    "points": "#C7DAE8",
    "trajectory": "#F92A82",
}

In [ ]:
UNIT_CUBE_VERTICES = (
    np.array(
        [
            (1, 1, 1),
            (1, 1, -1),
            (1, -1, 1),
            (1, -1, -1),
            (-1, 1, 1),
            (-1, 1, -1),
            (-1, -1, 1),
            (-1, -1, -1),
        ]
    )
    * 0.5
)


UNIT_CUBE_LINES_IDXS = np.array(
    [
        [0, 1],
        [0, 2],
        [0, 4],
        [1, 3],
        [1, 5],
        [2, 3],
        [2, 6],
        [3, 7],
        [4, 5],
        [4, 6],
        [5, 7],
        [6, 7],
    ]
)


PLOTTING_COLORS = {
    "wall": "#FBFAF5",
    "door": "#F7C59F",
    "window": "#53F4FF",
    "bbox": "#CC3FD1",
    "points": "#C7DAE8",
    "trajectory": "#F92A82",
}

def language_to_bboxes(entities):
    """
    Args:
        entities: List[BaseEntity].
    """
    box_definitions = []
    # lookup table
    lookup = {}

    for entity in entities:

        entity_id = int(entity.params["id"])
        class_name = entity.COMMAND_STRING[5:]  # remove "make_"

        if entity.COMMAND_STRING == "make_wall":
            height = entity.params["height"]
            thickness = 0.0
            # corners
            corner_a = np.array(
                [
                    entity.params["a_x"],
                    entity.params["a_y"],
                    entity.params["a_z"],
                ]
            )
            corner_b = np.array(
                [
                    entity.params["b_x"],
                    entity.params["b_y"],
                    entity.params["b_z"],
                ]
            )
            length = np.linalg.norm(corner_a - corner_b)

            direction = corner_b - corner_a
            angle = np.arctan2(direction[1], direction[0])
            lookup[entity_id] = {**entity.params, "angle": angle}

            centre = (corner_a + corner_b) * 0.5 + np.array([0, 0, 0.5 * height])
            scale = np.array([length, thickness, height])
            rotation = Rotation.from_rotvec([0, 0, angle]).as_matrix()

        elif entity.COMMAND_STRING in {"make_door", "make_window"}:

            # Find valid wall pointer
            # NOTE: this part differs from the original implementation of this function.
            for key in ["wall_id", "wall0_id", "wall1_id"]:
                wall_id = entity.params.get(key, None)
                wall = lookup.get(wall_id, None)
                if wall is not None:
                    break
            if wall is None:
                continue
            angle, thickness = wall["angle"], wall["thickness"]

            centre = np.array(
                [
                    entity.params["position_x"],
                    entity.params["position_y"],
                    entity.params["position_z"],
                ]
            )
            rotation = Rotation.from_rotvec([0, 0, angle]).as_matrix()
            scale = np.array(
                [
                    entity.params["width"],
                    thickness,
                    entity.params["height"],
                ]
            )

        elif entity.COMMAND_STRING == "make_bbox":

            centre = np.array(
                [
                    entity.params["position_x"],
                    entity.params["position_y"],
                    entity.params["position_z"],
                ]
            )
            rotation = Rotation.from_rotvec([0, 0, entity.params["angle_z"]]).as_matrix()
            scale = np.array(
                [
                    entity.params["scale_x"],
                    entity.params["scale_y"],
                    entity.params["scale_z"],
                ]
            )
            class_name = entity.params["class"]

        box = {
            "id": entity_id,
            "cmd": entity.COMMAND_STRING,
            "class": class_name,
            "centre": centre,
            "rotation": rotation,
            "scale": scale,
        }
        box_definitions.append(box)

    return box_definitions


def plot_box_wireframe(box):
    box_verts = UNIT_CUBE_VERTICES * box["scale"]
    box_verts = (box["rotation"] @ box_verts.T).T
    box_verts = box_verts + box["centre"]

    lines_x = []
    lines_y = []
    lines_z = []
    for pair in UNIT_CUBE_LINES_IDXS:
        for idx in pair:
            lines_x.append(box_verts[idx, 0])
            lines_y.append(box_verts[idx, 1])
            lines_z.append(box_verts[idx, 2])
        lines_x.append(None)
        lines_y.append(None)
        lines_z.append(None)

    if box["cmd"] == "make_bbox":
        class_name = f"bbox_{box['class']}"
        plot_color = PLOTTING_COLORS["bbox"]
    else:  # wall/door/window
        class_name = box["class"]
        plot_color = PLOTTING_COLORS[class_name]

    wireframe = go.Scatter3d(
        x=lines_x,
        y=lines_y,
        z=lines_z,
        mode="lines",
        name=f"{class_name}_{box['id']}",
        line={
            "color": plot_color,
            "width": 10,
        },
    )

    return wireframe


def plot_point_cloud(point_cloud, max_points_to_plot=50_000):
    if len(point_cloud) > max_points_to_plot:
        print(
            f"The number of points ({len(point_cloud)}) exceeds the maximum that can be reliably plotted."
        )
        print(f"Randomly subsampling {max_points_to_plot} points for the plot.")
        sampled = np.random.choice(len(point_cloud), max_points_to_plot, replace=False)
        point_cloud = point_cloud[sampled]

    return go.Scatter3d(
        x=point_cloud[:, 0],
        y=point_cloud[:, 1],
        z=point_cloud[:, 2],
        mode="markers",
        name="Semi-dense Point Cloud",
        marker={
            "size": 1.0,
            "opacity": 0.3,
            "color": PLOTTING_COLORS["points"],
        },
    )


# Main plotting function
def plot_3d_scene(
    language_sequence=None,
    point_cloud=None,
    max_points_to_plot=50_000,
    fig_width=1000,
):

    traces = []
    if point_cloud is not None:
        traces.append(plot_point_cloud(point_cloud, max_points_to_plot))

    if language_sequence is not None:
        boxes = language_to_bboxes(language_sequence.entities)
        for box in boxes:
            traces.append(plot_box_wireframe(box))

    assert traces, "Nothing to visualize."
    fig = go.Figure(data=traces)
    fig.update_layout(
        template="plotly_dark",
        scene={
            "xaxis": {"showticklabels": False, "title": ""},
            "yaxis": {"showticklabels": False, "title": ""},
            "zaxis": {"showticklabels": False, "title": ""},
        },
        width=fig_width,
        height=fig_width // 2,
        scene_aspectmode="data",
        hoverlabel={"namelength": -1},
    )
    fig.show()

In [ ]:
!dir

CODE_OF_CONDUCT.md  environment.yaml  inference.ipynb  models	  src
CONTRIBUTING.md     imgs	      LICENSE	       README.md




---

##Upload model

The model can be dowloaded here:

https://www.projectaria.com/scenescript/

In [ ]:
!mkdir models

mkdir: cannot create directory ‘models’: File exists


In [ ]:
import shutil
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  shutil.move(fn, 'models/')


Saving scenescript_model_ase.ckpt to scenescript_model_ase.ckpt


In [ ]:
ckpt_path = f"models/scenescript_model_ase.ckpt"  # TODO: path to downloaded model checkpoint
model_wrapper = SceneScriptWrapper.load_from_checkpoint(ckpt_path).cuda()



---

## Run on a file from the test dataset

In [ ]:
import os
import requests
from tqdm import tqdm

def download_file(url, filename):
    """
    Downloads a file from a URL with a progress bar.
    """
    response = requests.get(url, stream=True)
    response.raise_for_status()  # Raise an exception for bad status codes

    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 KB
    progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True)

    with open(filename, 'wb') as file:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)

    progress_bar.close()

    if total_size != 0 and progress_bar.n != total_size:
        print("ERROR, something went wrong")

# Create the directory if it doesn't exist
os.makedirs("/content/scenescript/dataset", exist_ok=True)

# Download the file
file_url = "https://www.projectaria.com/async/sample/download/?bucket=ase&filename=ase_examples.zip"
download_path = "/content/scenescript/dataset/ase_examples.zip"
download_file(file_url, download_path)


100%|██████████| 283M/283M [00:20<00:00, 13.7MiB/s]


In [ ]:
!unzip -o /content/scenescript/dataset/ase_examples.zip -d /content/scenescript/dataset/

Archive:  /content/scenescript/dataset/ase_examples.zip
   creating: /content/scenescript/dataset/0/
  inflating: /content/scenescript/dataset/0/ase_scene_language.txt  
  inflating: /content/scenescript/dataset/0/object_instances_to_classes.json  
  inflating: /content/scenescript/dataset/0/trajectory.csv  
  inflating: /content/scenescript/dataset/0/semidense_points.csv.gz  
  inflating: /content/scenescript/dataset/0/semidense_observations.csv.gz  
   creating: /content/scenescript/dataset/0/rgb/
  inflating: /content/scenescript/dataset/0/rgb/vignette0000000.jpg  
  inflating: /content/scenescript/dataset/0/rgb/vignette0000001.jpg  
  inflating: /content/scenescript/dataset/0/rgb/vignette0000002.jpg  
  inflating: /content/scenescript/dataset/0/rgb/vignette0000003.jpg  
  inflating: /content/scenescript/dataset/0/rgb/vignette0000004.jpg  
  inflating: /content/scenescript/dataset/0/rgb/vignette0000005.jpg  
  inflating: /content/scenescript/dataset/0/rgb/vignette0000006.jpg  
  inf

In [ ]:
point_cloud_path = "/content/scenescript/dataset/1/semidense_points.csv.gz"  # TODO: path to semidense point cloud
point_cloud_obj = PointCloud.load_from_file(point_cloud_path)

Kept 258118 points after filtering!


In [ ]:
lang_seq = model_wrapper.run_inference(
    point_cloud_obj.points,
    nucleus_sampling_thresh=0.05,  # 0.0 is argmax, 1.0 is random sampling
    verbose=True,
)

Time taken for input encoding: 2.499s
Time taken for autoregressive sampling: 2.737s


In [ ]:
plot_3d_scene(
    lang_seq,
    point_cloud_obj.points,
    max_points_to_plot=50_000,
    fig_width=1100,
)

The number of points (258118) exceeds the maximum that can be reliably plotted.
Randomly subsampling 50000 points for the plot.




---

##Upload custom data

In [ ]:
import os
from google.colab import files

# Create the custom_data directory if it doesn't exist
custom_data_dir = "custom_data"
os.makedirs(custom_data_dir, exist_ok=True)

# Allow file uploads to the custom_data directory
uploaded = files.upload()

for filename in uploaded.keys():
  print(f'User uploaded file "{filename}" with length {len(uploaded[filename])} bytes')
  # Move the uploaded file to the custom_data directory
  destination_path = os.path.join(custom_data_dir, filename)
  with open(destination_path, "wb") as f:
    f.write(uploaded[filename])
  print(f"File '{filename}' moved to '{destination_path}'")


Saving mesh_vertices_generated.csv.gz to mesh_vertices_generated.csv.gz
User uploaded file "mesh_vertices_generated.csv.gz" with length 22947638 bytes


In [ ]:
point_cloud_path_test = "/content/scenescript/custom_data/mesh_vertices_generated.csv.gz"  # TODO: path to semidense point cloud
point_cloud_obj_test = PointCloud.load_from_file(point_cloud_path_test)

Kept 746371 points after filtering!


In [ ]:
lang_seq_test = model_wrapper.run_inference(
    point_cloud_obj_test.points,
    nucleus_sampling_thresh=0.1,  # 0.0 is argmax, 1.0 is random sampling
    verbose=True,
)

Time taken for input encoding: 1.229s
Time taken for autoregressive sampling: 5.062s


In [ ]:
plot_3d_scene(
    lang_seq_test,
    point_cloud_obj_test.points,
    max_points_to_plot=50_000,
    fig_width=1800,
)

The number of points (746371) exceeds the maximum that can be reliably plotted.
Randomly subsampling 50000 points for the plot.
